In [1]:
!pip install simpletransformers

In [2]:
import json
with open(r"train.json", "r") as read_file:
    train = json.load(read_file)

In [3]:
train

[{'context': 'Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Who is the author of the Mistborn series?',
    'answers': [{'text': 'Brandon Sanderson', 'answer_start': 71}]}]},
 {'context': 'The first series, published between 2006 and 2008, consists of The Final Empire,The Well of Ascension, and The Hero of Ages.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'When was the series published?',
    'answers': [{'text': 'between 2006 and 2008', 'answer_start': 28}]},
   {'id': '00003',
    'is_impossible': False,
    'question': 'What are the three books in the series?',
    'answers': [{'text': 'The Final Empire, The Well of Ascension, and The Hero of Ages',
      'answer_start': 63}]},
   {'id': '00004',
    'is_impossible': True,
    'question': 'Who is the main character in the series?',
    'answers': []}]}]

In [4]:
with open(r"test.json", "r") as read_file:
    test = json.load(read_file)

In [5]:
test

[{'context': 'The series primarily takes place in a region called the Final Empire on a world called Scadrial, where the sun and sky are red, vegetation is brown, and the ground is constantly being covered under black volcanic ashfalls.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Where does the series take place?',
    'answers': [{'text': 'region called the Final Empire', 'answer_start': 38},
     {'text': 'world called Scadrial', 'answer_start': 74}]}]},
 {'context': '"Mistings" have only one of the many Allomantic powers, while "Mistborns" have all the powers.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'How many powers does a Misting possess?',
    'answers': [{'text': 'one', 'answer_start': 21}]},
   {'id': '00003',
    'is_impossible': True,
    'question': 'What are Allomantic powers?',
    'answers': []}]}]

In [6]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [7]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [8]:
!pip install wandb

In [9]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [10]:
#Initialize the model
import torch
from transformers.data.processors.squad import SquadFeatures
torch.serialization.add_safe_globals([SquadFeatures])  # Add SquadFeatures to safe globals

model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
### Remove output folder
!rm -rf outputs

In [11]:
# Train the model
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 4/4 [00:00<00:00, 36235.89it/s]


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

...

Running Epoch 9 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(10,
 {'global_step': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'correct': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'similar': [3, 2, 3, 3, 3, 3, 3, 3, 3, 3],
  'incorrect': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  'train_loss': [4.671865463256836,
   4.673550128936768,
   4.2130913734436035,
   3.754096031188965,
   3.0929880142211914,
   2.66438627243042,
   2.4954700469970703,
   2.1752219200134277,
   2.0473413467407227,
   1.953507423400879],
  'eval_loss': [-0.10426206141710281,
   -0.17805851995944977,
   -0.2567831575870514,
   -0.3335762023925781,
   -0.40983691811561584,
   -0.48222753405570984,
   -0.5441485643386841,
   -0.592784583568573,
   -0.6259663105010986,
   -0.6428844332695007]})

In [12]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
result

{'correct': 0, 'similar': 3, 'incorrect': 0, 'eval_loss': -0.6428844332695007}

In [14]:
# Make predictions with the model
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]

In [15]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4549.14it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['power and', 'great power and', 'power']}]


In [16]:
### save the model
# Save Hugging Face model & tokenizer manually
model.model.save_pretrained("qa_model/")
model.tokenizer.save_pretrained("qa_model/")


('qa_model/tokenizer_config.json',
 'qa_model/special_tokens_map.json',
 'qa_model/vocab.txt',
 'qa_model/added_tokens.json')

In [18]:
!zip -r qa_model.zip qa_model


updating: qa_model/ (stored 0%)
updating: qa_model/config.json (deflated 47%)
updating: qa_model/tokenizer_config.json (deflated 75%)
updating: qa_model/special_tokens_map.json (deflated 42%)
updating: qa_model/model.safetensors (deflated 7%)
updating: qa_model/vocab.txt (deflated 49%)
